# 하이브리드 암호 시스템

### 🔐 하이브리드 암호 시스템 구조 요약:

- **비대칭키 암호 (공개키/개인키)**: 세션 키를 안전하게 전달하는 데 사용
    - 예: RSA, ECC 등
    - 데이터량이 많거나 속도가 중요한 실제 메시지 암호화에는 비효율적
- **대칭키 암호 (세션 키)**: 본문 메시지를 암호화하는 데 사용
    - 예: AES, ChaCha20 등
    - 빠르고 효율적이지만 키 분배 문제가 있음

---

### 💡 정리:

- **세션 키 = 대칭 키**
- 하이브리드 암호 방식에서는 세션 키를 임의로 생성하여 **공개키로 암호화하여 전송**하고,
- 수신자는 **개인키로 복호화**하여 세션 키를 얻은 뒤, 이후 통신은 세션 키(AES 등)로 암호화된 데이터를 주고받습니다.

---

시험 대비용 포인트:

> “하이브리드 암호 시스템에서 세션 키는 대칭 암호 알고리즘에 사용되는 임시 키로, 공개키 암호화 방식으로 안전하게 전달된다.” ✅
>

### 🧠 전제 개념 정리

| 내용 | 사용 키 | 목적 |
| --- | --- | --- |
| 🔒 데이터 암호화 | 대칭키(K_AB) | 빠르고 효율적인 암호화 |
| 🔐 세션키(K_AB) 전달 | 수신자의 공개키(PU_B)로 암호화 | 안전한 키 교환 |
| 🔓 세션키 복호화 | 수신자의 개인키(PR_B) | 수신자가 세션키 획득 |

---

## 💻 파이썬 조각 예시: 하이브리드 암호 시스템

In [3]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import base64

# 1. 수신자 B의 공개/개인 키 생성 (비대칭)
receiver_key = RSA.generate(2048)
public_key_B = receiver_key.publickey()
cipher_rsa_enc = PKCS1_OAEP.new(public_key_B)     # 암호화용 (공개키)
cipher_rsa_dec = PKCS1_OAEP.new(receiver_key)     # 복호화용 (개인키)

# 2. 세션 키 생성 (대칭키)
session_key = get_random_bytes(16)  # AES-128

# 3. 송신자가 메시지를 AES로 암호화
message = b"Hello, this is a secret message"
cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(message)

# 4. 송신자가 세션 키를 수신자의 공개키로 암호화
encrypted_session_key = cipher_rsa_enc.encrypt(session_key)

# ===> 송신자가 전송하는 것
# ciphertext, encrypted_session_key, nonce, tag
print(ciphertext)
print(encrypted_session_key)


b'm\xd0M\xcacl\xf7\x06\xbc\x83"\x87\xea\xf1\x99\xeb\x94\xedc\x00qG\xfc?Tp\xdaf \x83.'
b'/\xc3\xedu\\\xf9\xee\x828;\xe7\x9f\xad\xc5\xe6_\x10_\x0f}b\xeeA\xa0\x15*}\xf0\x8e\xefz\xd6\x84={\xfd\xb0\x8bH\x84\xd2\x1dSsX\xc4e\x13\xd6a+\xac\xd4H\xf5\xad\xfa}r\x98\x92\x80)\xa1y{\xeb\xb2_(\xe1\x11\xd5\xcc\x956\xdf\x0c\xa9\xf7wu1m\xf7\xbcB\x85\x84h\xe0\x99\x15\x9a\xeb\x824DPf\x0b\xbe\xb4\xa3e\x05\x05\x02q\x13qh\xd9\n\x84\xfe\xb7\x10\x87\xfaR\x82b\x83Z\xfe\xad[\xe0\x99iv\x8d\x97\x8e\x02\x17\xade\xfc355\x8ef\x7f\x0e\x13A(\x11O\x84\xc0e\x94|\xac\xa9\xa1f#+GY\r\x89e=\xac[\x816?\x95\xc1\xa3<\xebH\x84\xaa\xba\x81\x86vfa\x0e!tfUW\xb9]\xf9\xb5\xd8\x9cVA\r\x88b\xad@\x13\xd8L\x02`C?\x8a\xa7\x00\x15\xc95AeB@A@\x8b\xc0\xfa_\xecto\xc1_\x92\xef\xe7\x94\xa8J]g\x1a\xee\x7f=\x97\xb6\xc3\x1d\xfc3\xe6B\x9f'


### 수신자(B) 복호화 과정

In [4]:

# 1. 수신자가 개인키로 세션키 복호화
decrypted_session_key = cipher_rsa_dec.decrypt(encrypted_session_key)

# 2. 복호화한 세션키로 메시지 복호화
cipher_aes_receiver = AES.new(decrypted_session_key, AES.MODE_EAX, nonce=cipher_aes.nonce)
plaintext = cipher_aes_receiver.decrypt_and_verify(ciphertext, tag)

print(plaintext.decode())  # Hello, this is a secret message


Hello, this is a secret message


## 🔁 전체 흐름 요약

| 단계 | 행위자 | 방식 | 사용 키 |
| --- | --- | --- | --- |
| ① 세션 키 생성 | 송신자 A | 대칭키 준비 | K_AB |
| ② 메시지 암호화 | 송신자 A | 대칭키 AES | K_AB |
| ③ 세션 키 암호화 | 송신자 A | 수신자 공개키 | **PU_B** |
| ④ 세션 키 복호화 | 수신자 B | 수신자 개인키 | **PR_B** |
| ⑤ 메시지 복호화 | 수신자 B | 대칭키 AES | K_AB |

---

## 📌 시험용 키워드 요약

- **대칭키로는 메시지를 암호화**
- **비대칭키는 세션키를 안전하게 전달**
- **공개키로 암호화 → 개인키로 복호화**
- **수신자의 공개키로 암호화, 수신자의 개인키로 복호화**